In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import sqlite3
import mysql.connector as cnt
import time


In [2]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [3]:
bs = BeautifulSoup(source, 'html.parser')

butai_nuorodos = []

linkai = bs.find_all('h3')
for linkas in linkai:
    # print('::REM::')
    l = linkas.find('a')
    if l != None:
        # print('error')
        # print(l['href'])
        butai_nuorodos.append(l['href'])
        
print(len(butai_nuorodos))
print(butai_nuorodos)
  

25
['https://www.aruodas.lt/butai-vilniuje-snipiskese-verkiu-g-parduodamas-jaukus-ir-sviesus-butas-butas-1-3416089/', 'https://www.aruodas.lt/butai-palangoje-sventojoje-prieplaukos-g-parduodami-isskirtiniai-apartamentai-prie-1-3488093/', 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/', 'https://www.aruodas.lt/butai-vilniuje-fabijoniskese-vetrungiu-g-parduodamas-jaukus-siltas-sviesus-1-3486583/', 'https://www.aruodas.lt/butai-vilniuje-senamiestyje-traku-g-senamiestyje-traku-g-praduodamas-sviesus-1-3446254/', 'https://www.aruodas.lt/butai-vilniuje-vilkpedeje-savanoriu-pr-parduodamas-stilingo-dizaino-itin-kokybiskai-1-3470443/', 'https://www.aruodas.lt/butai-vilniuje-snipiskese-krokuvos-g-moderniai-ir-kokybiskai-irengtas-kambariu-1-3484415/', 'https://www.aruodas.lt/butai-kaune-silainiuose-v-kavolio-g-butai-tiesiogiai-is-statytojo-silainiuose-1-3480325/', 'https://www.aruodas.lt/butai-kaune-sanciuose-a-juozapaviciaus-p

In [4]:
kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
for k in kitas:
    # print(k.text.strip())
    if k.text.strip() == '»':
        kitas_psl = k['href']
        # print(kitas_psl)
next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'

print(next_nuoroda)

https://www.aruodas.lt//butai/puslapis/2/?FOwnerDbId0=1&FOwnerDbId2=1


In [5]:
driver = webdriver.Chrome(options=opcijos)
for nuoroda in butai_nuorodos[0:1]:
    print(nuoroda)
    
    
    # url = nuoroda
    url = 'https://www.aruodas.lt/butai-palangoje-kretingos-g-isskirtinis-kambariu-butas-palangoje-1-3488061/'
    # url = 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-dvieju-kambariu-butas-naujame-1-3481853/'
    # url = 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/'
    # url = 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/'
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    
driver.close()

https://www.aruodas.lt/butai-vilniuje-snipiskese-verkiu-g-parduodamas-jaukus-ir-sviesus-butas-butas-1-3416089/


In [12]:
# Bandymas papildyti lentele
def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists Aruodas
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists Aruodas
    (
        miestas text,
        rajonas text,
        gatve text,
        kaina numeric,
        kv_kaina numeric
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

createMainTable('aruod.db')

OperationalError: database is locked

In [7]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names
# driver = webdriver.Chrome(options=opcijos)
# for nuoroda in butai_nuorodos[0:5]:
# print(nuoroda)
cols_i_have = getColNamesList(table='Aruodas', dbname='aruod.db')
duomenys = {}
for col in cols_i_have:
    duomenys[col] = None

print(duomenys)

duomenys['nuoroda'] = nuoroda

url = nuoroda
# url = 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-dvieju-kambariu-butas-naujame-1-3481853/'
# driver.get(url)
# time.sleep(2)
# source = driver.page_source

bs_buto = BeautifulSoup(source, 'html.parser')

gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
gyvenviete_lst = gyvenviete.split(',')
if len(gyvenviete_lst) == 4:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    print(miestas)
    rajonas = gyvenviete_lst[1].strip()
    duomenys['rajonas'] = rajonas
    print(rajonas)
    gatve = gyvenviete_lst[2].strip()
    duomenys['gatve'] = gatve
    print(gatve)
if len(gyvenviete_lst) == 3:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    print(miestas)
    gatve = gyvenviete_lst[1].strip()
    duomenys['gatve'] = gatve

    
kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
duomenys['kaina'] = kaina
print(kaina)

kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
# print(kv_kaina_txt)
kv_kaina = int(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', ''))
duomenys['kv_kaina'] = kv_kaina
print(kv_kaina)


info_keys = []
info_values = []
info_k = bs_buto.find('dl', {'class':'obj-details'}).find_all('dt')
for i in info_k:
    # print(i.text.strip())
    info_keys.append(i.text.strip())
    a = i.text.strip()
    # if a not in duomenys.keys():
    #     print(a)
print(info_keys)
print(len(info_keys))

# info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('span',{'class':'fieldValueContainer'})
info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('dd')
for i in info_v:
    # print(i.text.strip())
    info_values.append(i.text.strip())
print(info_values)
print(len(info_values))

for k, v in zip(info_keys, info_values):
    duomenys[k] = v

print(duomenys)
col_i_need = list(duomenys.keys())
print(col_i_need)

data = tuple(duomenys.values())
data1 = []
data1.append(data)

# SDB = sqlite3.connect('Aruodas_test.db')  # jei neegzistuoja db , bus sukurta nauja db
# Cs = SDB.cursor()

# #  if not exists - tikrina ar jau sukurta DB
# sql = '''create table if not exists Aruodas
# (
# miestas text not null,
# rajonas text not null,
# gatve text not null,
# kaina numeric not null,
# kvadatoKaina numeric not null,
# plotas text not null,
# kambariai text not null,
# metai text not null,
# aukstas text not null,
# aukstuSk text not null,
# namoNr text not null,
# butoNr text not null,
# pastatoTipas text not null,
# sildymas text not null,
# irengimas text not null,
# energetineKlase text not null,
# ypatybes text not null,
# papildomosPatalpos text not null,
# papildomaIranga text not null,
# apsauga text not null,
# unikalusNr text not null,
# nuoroda text not null
# )
# '''
# Cs.execute(sql)

# sql_template = '''insert into Aruodas values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
# Cs.executemany(sql_template, data1)

# SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

# SDB.close()
# print('data written to DB')     
    
    
    
# driver.close()
print('Baigta!!!')

# .text
# .contents
# .content

{'miestas': None, 'rajonas': None, 'gatve': None, 'kaina': None, 'kv_kaina': None}
Palanga
135000
3712
['Namo numeris:', 'Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Aukštų sk.:', 'Metai:', 'Pastato tipas:', 'Šildymas:', 'Įrengimas:', 'Ypatybės:', 'Papildomos patalpos:', 'Papildoma įranga:', 'Apsauga:']
13
['51', '36,37 m²', '2', '1', '4', '2013', 'Mūrinis', 'Centrinis kolektorinis', 'Įrengtas', 'Nauja kanalizacija\nNauja elektros instaliacija\nUždaras kiemas\nInternetas\nVirtuvė sujungta su kambariu', 'Balkonas\nRūsys\nVieta automobiliui', 'Skalbimo mašina\nSu baldais\nŠaldytuvas\nVirtuvės komplektas\nViryklė\nPlastikiniai vamzdžiai\nDušo kabina', 'Šarvuotos durys\nKodinė laiptinės spyna']
13
{'miestas': 'Palanga', 'rajonas': None, 'gatve': 'Kretingos g.', 'kaina': 135000, 'kv_kaina': 3712, 'nuoroda': 'https://www.aruodas.lt/butai-vilniuje-snipiskese-verkiu-g-parduodamas-jaukus-ir-sviesus-butas-butas-1-3416089/', 'Namo numeris:': '51', 'Plotas:': '36,37 m²', 'Kambarių sk.:': '2', 'Aukštas

In [8]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

col_i_have = getColNamesList(table='Aruodas', dbname='Aruodas_test.db')
print(col_i_have)

['miestas', 'rajonas', 'gatve', 'kaina', 'kvadatoKaina', 'plotas', 'kambariai', 'metai', 'aukstas', 'aukstuSk', 'namoNr', 'butoNr', 'pastatoTipas', 'sildymas', 'irengimas', 'energetineKlase', 'ypatybes', 'papildomosPatalpos', 'papildomaIranga', 'apsauga', 'unikalusNr', 'nuoroda']


In [15]:
def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True

In [9]:
# Bandymas papildyti lentele
def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists Aruodas
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists Aruodas
    (
        miestas text not null,
        rajonas text not null,
        gatve text not null,
        kaina numeric not null,
        kv_kaina numeric not null
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(3)
    c.close()
    db.close()
    return True

# createMainTable('aruod.db')
col_i_have = getColNamesList(table='Aruodas', dbname='aruod.db')
print(col_i_have)
print(col_i_need)


['miestas', 'rajonas', 'gatve', 'kaina', 'kv_kaina']
['miestas', 'rajonas', 'gatve', 'kaina', 'kv_kaina', 'nuoroda', 'Namo numeris:', 'Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Aukštų sk.:', 'Metai:', 'Pastato tipas:', 'Šildymas:', 'Įrengimas:', 'Ypatybės:', 'Papildomos patalpos:', 'Papildoma įranga:', 'Apsauga:']


In [10]:
alterTableAddColumn(col_i_need, col_i_have, 'Aruodas', 'aruod.db')
print(getColNamesList('Aruodas', 'aruod.db'))

['miestas', 'rajonas', 'gatve', 'kaina', 'kv_kaina', 'nuoroda', 'Namo numeris:', 'Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Aukštų sk.:', 'Metai:', 'Pastato tipas:', 'Šildymas:', 'Įrengimas:', 'Ypatybės:', 'Papildomos patalpos:', 'Papildoma įranga:', 'Apsauga:']


In [11]:
def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


writeMany(data, 'Aruodas', 'aruod.db')

IntegrityError: NOT NULL constraint failed: Aruodas.rajonas

In [ ]:
# dėl laiko apribojimo

# t = now()
# now() - t
# DeltaTimeStamp
# Timedelta
# .total_seconds

# https://www.geeksforgeeks.org/python-difference-between-two-dates-in-minutes-using-datetime-timedelta-method/

In [5]:
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [6]:
bs = BeautifulSoup(source, 'html.parser')

cities = []

miestai = bs.find('div', {'id':'searchFormField_FRegion'}).find_all('label', {'class':'dropDownLabel'})
for miestas in miestai:
    i  = miestas.text.strip()
    if i != '':
        # print(i)
        cities.append(i)
        
print(cities)
print(len(cities))

cities_for_link = ['vilniuje', 'kaune', 'klaipedoje', 'siauliuose', 'panevezyje', 'alytuje', 'palangoje',
                   'akmeneje', 'alytaus-rajone', 'anysciuose', 'birstone', 'birzuose', 'druskininkuose',
                   'elektrenuose', 'ignalinoje', 'jonavoje', 'joniskyje', 'jurbarke', 'kaisiadoryse',
                   'kalvarijoje', 'kauno-rajone', 'kazlu-rudoje', 'kedainiuose', 'kelmeje', 'klaipedos-rajone',
                   'kretingoje', 'kupiskyje', 'lazdijuose', 'marijampoleje', 'mazeikiuose', 'moletuose',
                   'neringoje', 'pagegiuose', 'pakruojyje', 'panevezio-rajone', 'pasvalyje', 'plungeje',
                   'prienuose', 'radvilisyje', 'raseiniuose', 'rietave', 'rokiskyje', 'skuode', 'sakiuose',
                   'salcininkuose', 'siauliu-rajone', 'silaleje', 'siluteje', 'sirvintuose', 'svencionyse',
                   'taurageje', 'telsiuose', 'trakuose', 'ukmergeje', 'utenoje', 'varenoje', 'vilkaviskyje',
                   'vilniaus-rajone', 'visagine', 'zarasuose']

print(cities_for_link)
print(len(cities_for_link))
    

['Vilnius', 'Kaunas', 'Klaipėda', 'Šiauliai', 'Panevėžys', 'Alytus', 'Palanga', 'Akmenės r.', 'Alytaus r.', 'Anykščių r.', 'Birštono sav.', 'Biržų r.', 'Druskininkų sav.', 'Elektrėnų sav.', 'Ignalinos r.', 'Jonavos r.', 'Joniškio r.', 'Jurbarko r.', 'Kaišiadorių r.', 'Kalvarijos sav.', 'Kauno r.', 'Kazlų Rūdos sav.', 'Kėdainių r.', 'Kelmės r.', 'Klaipėdos r.', 'Kretingos r.', 'Kupiškio r.', 'Lazdijų r.', 'Marijampolės sav.', 'Mažeikių r.', 'Molėtų r.', 'Neringos sav.', 'Pagėgių sav.', 'Pakruojo r.', 'Panevėžio r.', 'Pasvalio r.', 'Plungės r.', 'Prienų r.', 'Radviliškio r.', 'Raseinių r.', 'Rietavo sav.', 'Rokiškio r.', 'Skuodo r.', 'Šakių r.', 'Šalčininkų r.', 'Šiaulių r.', 'Šilalės r.', 'Šilutės r.', 'Širvintų r.', 'Švenčionių r.', 'Tauragės r.', 'Telšių r.', 'Trakų r.', 'Ukmergės r.', 'Utenos r.', 'Varėnos r.', 'Vilkaviškio r.', 'Vilniaus r.', 'Visagino sav.', 'Zarasų r.']
60
['vilniuje', 'kaune', 'klaipedoje', 'siauliuose', 'panevezyje', 'alytuje', 'palangoje', 'akmeneje', 'alytaus-

In [7]:
for idx, city in enumerate(cities):
    print(idx, city)

0 Vilnius
1 Kaunas
2 Klaipėda
3 Šiauliai
4 Panevėžys
5 Alytus
6 Palanga
7 Akmenės r.
8 Alytaus r.
9 Anykščių r.
10 Birštono sav.
11 Biržų r.
12 Druskininkų sav.
13 Elektrėnų sav.
14 Ignalinos r.
15 Jonavos r.
16 Joniškio r.
17 Jurbarko r.
18 Kaišiadorių r.
19 Kalvarijos sav.
20 Kauno r.
21 Kazlų Rūdos sav.
22 Kėdainių r.
23 Kelmės r.
24 Klaipėdos r.
25 Kretingos r.
26 Kupiškio r.
27 Lazdijų r.
28 Marijampolės sav.
29 Mažeikių r.
30 Molėtų r.
31 Neringos sav.
32 Pagėgių sav.
33 Pakruojo r.
34 Panevėžio r.
35 Pasvalio r.
36 Plungės r.
37 Prienų r.
38 Radviliškio r.
39 Raseinių r.
40 Rietavo sav.
41 Rokiškio r.
42 Skuodo r.
43 Šakių r.
44 Šalčininkų r.
45 Šiaulių r.
46 Šilalės r.
47 Šilutės r.
48 Širvintų r.
49 Švenčionių r.
50 Tauragės r.
51 Telšių r.
52 Trakų r.
53 Ukmergės r.
54 Utenos r.
55 Varėnos r.
56 Vilkaviškio r.
57 Vilniaus r.
58 Visagino sav.
59 Zarasų r.


In [8]:
pasirink = input('Įvesk mieto numerius, atskirtus kableliais, iš kurių nori gauti skelbimus: ')
pasirink_pvz = '8,23,35' # Alytus, Kelme, Pasvalys
pas = [int(i) for i in pasirink.split(',')]
pasirinktas_miestas_link = []
for i in pas:
    pasirinktas_miestas_link.append(cities_for_link[i])
print(pasirinktas_miestas_link)

pasirinktas_miestas = []

['kaune', 'klaipedoje', 'alytuje']


In [ ]:
url = f'https://www.aruodas.lt/butai/{pasirinktas_miestas}/puslapis/2/?FOwnerDbId0=1&FOwnerDbId2=1'